In [100]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle 

In [101]:
##import tensorflow as tf
##tf.config.run_functions_eagerly(True)


In [102]:
## Load the dataset
data= pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [103]:
## Preprocess the data
### Drop irrelevent columns
data= data.drop(['RowNumber','CustomerId','Surname'], axis=1)     ## Axis=1 means column wise.
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [104]:
## Encode categorical variables (this is one of the encoding technique)
label_encoder_gender = LabelEncoder()
data['Gender']= label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


For geography column we have 3 categories: Here if we go ahead and apply labelEncoder() then france=0, Spain=1, Germany=2,and when this types of numbers are specifically coming up, then there is a problem because, when we assign a value of germany to 2, At the end of the day ANN is all about numerical and calculations, since the number/label of germany is 2, it will just consider that Germany is grater than spain, or spain is greater than france, and this should not happen, so for this particular case, we will not use label encoder instead we will use OneHotEncoding. OneHotEncoding will give value with respect to 0s and 1s.


In [105]:
## OneHotEncoder 'Geography'
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [106]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [107]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [108]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [109]:
## Combine one hot encoder columns with the original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


What is pickle?
pickle is a built-in Python module used to save Python objects to a file.
Think of it like "saving your work", but for Python variables, models, encoders, etc.
You can also use it to load that work back later, just like saving and opening a game.

_____________________________________________________________________________________________________________
What does dump mean?
pickle.dump(object, file) means:
👉 "Put this object into that file using pickle format."Think of dump like "dumping the contents of a backpack into a locker" — saving something to use later.

_____________________________________________________________________________________________________________
What does 'wb' mean?
'wb' stands for:
w → "write" (you want to write to the file)
b → "binary" (pickle needs binary format, not text). So 'wb' = write in binary mode.

_____________________________________________________________________________________________________________
What's happening?
label_encoder_gender and onehot_encoder_geo are probably sklearn encoders used in a machine learning project.
You’re saving them so you can use the exact same encoders later when you:

*Load a model

*Predict new data

*Keep consistent data preprocessing

_____________________________________________________________________________________________________________
Word:

pickle - Module to save/load Python objects

dump() - Save the object to a file

'wb' - Write in binary mode (for saving)

'rb' - Read in binary mode (for loading)

In [110]:
## Save the encoders and scaler.   
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file) 
  

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo, file)


In [111]:
## Divide the dataset into independent and dependent 
X=data.drop('Exited', axis=1)           ## Input features (independent variables)
y= data['Exited']                       ## Output/label (dependent variable)

## Split the data in training and testing sets
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()              ## Create a scaler object
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


What is "scale"?
To scale data means to change the values in your dataset so they are all within a similar range — without changing their relationships.

In simple words:

Scaling” means adjusting all the numbers in your dataset so they’re in the same range — like normalizing students' grades out of 10 even if one teacher used 100 marks and another used 50.


🎯 Why scale?

Imagine this data:
Feature with values:

Age (in years)	25, 30, 45

Salary (in ₹ rupees)	20,000, 50,000, 90,000

Now think about this:
These values are very different in scale.
If you give this to a machine learning model, it might think salary is "more important" just because the numbers are bigger — even if it’s not!
So we scale the data to bring all the features to a similar range, like -1 to +1 or 0 to 1.

_________________________________________________________________________________________________________________________


 Standard Scaling
You're using:

scaler = StandardScaler() - This does Standard Scaling, also called Z-score normalization. It converts each feature so that:

*The mean (average) becomes 0

*The standard deviation becomes 1

Formula:

scaled value = value − mean/standard deviation

​_______________________________________________________________________________________________________________________







scaler = StandardScaler():This creates a StandardScaler object from sklearn.preprocessing.

______________________________________________________________________________________________________________
Term:

scaler - A variable name for a scaling tool

StandardScaler() - A scikit-learn scaler that standardizes features

fit() - Learns the mean and std from data

transform() - Applies the learned scaling

fit_transform() - Shortcut to do both at once on training data

______________________________________________________________________________________________________________
What is scaler.fit_transform()?
This method does two things in one step:
fit() → calculates the mean and standard deviation of the training data.
transform() → uses those numbers to scale the data.

______________________________________________________________________________________________________________
fit_transform() does two jobs:
fit() — Looks at all the numbers in X_train, and calculates:
        *The mean (average) of each column
        *The standard deviation of each column.

transform() — Then uses that info to scale all the values like this.
 new_value = (value - mean) / standard_deviation

______________________________________________________________________________________________________________
X_test = scaler.transform(X_test):
Very important step.You do NOT fit again on test data.You just use the same mean and std learned from X_train to scale X_test.This keeps your model honest — you’re not “peeking” at the test set during training.

______________________________________________________________________________________________________________
Why Scaling is Important?
Some ML models (especially neural networks and gradient-based models) work much better and faster when all input features are scaled evenly.
Imagine you're comparing someone's weight (in kg) to their income (in ₹ rupees). One number is in hundreds, the other in lakhs — the model will get confused unless you scale them down to a similar range.


________________________________________________________________________________________________________________
Line of Code:

scaler = StandardScaler() - Create the scaler tool

fit_transform(X_train) - Learn scaling from training data + apply it

transform(X_test) - Apply the same scaling to test data (without re-learning)


In [112]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [113]:
with open('scaler.pkl','wb') as file:
      pickle.dump(scaler, file)

In [114]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


ANN Implementation

In [115]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

🧠 What is Dense in Keras?
In Keras (which is a tool to build neural networks), a Dense layer is the most basic and common type of layer.

💡 Simple Definition:
A Dense layer is a layer where every neuron is connected to every neuron in the previous layer.
That’s why it’s also called a fully connected layer.

✅ When do we use Dense?
1. In most deep learning models — especially in classification, regression, or the final output layer.

2. After extracting features (like from CNNs or LSTMs), you often pass them into Dense layers.
______________________________________________________________________________________________________________________

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard ->This line is importing tools from Keras, part of the TensorFlow library.

You are bringing in:
1. EarlyStopping
2. TensorBoard
These are both callbacks. Let’s explain everything word by word 👇

🧠 What are "callbacks" in Keras?
✅ Callback = "something that runs automatically during training". A callback is a tool in Keras that runs automatically during training to help monitor, stop, or save your model. Callbacks make your training smarter.

* Think of callbacks like smart assistants that:
1. Watch your model train
2. Take actions at the end of each epoch (like saving, stopping, printing, logging)


* Think of callbacks as helpers that Keras uses to monitor training and do things like:
1. Stop early if it’s not improving
2. Save the model
3. Log progress

They are like “custom instructions” that run at the end of each epoch during training.


Common Types of Callbacks:

Callback Name
EarlyStopping---->Stops training if the model stops improving

ModelCheckpoint---->Saves the best version of your model automatically

TensorBoard---->Lets you see graphs, logs, and training metrics in your browser

ReduceLROnPlateau---->Lowers learning rate if model stops improving
____________________________________________________________________________________________________________________________

🛑 What is EarlyStopping?
EarlyStopping is a callback that stops training early if the model’s performance stops getting better (like if validation loss doesn't improve). It helps: Save time and Prevent overfitting. EarlyStopping prevents wasting time training when the model has already done its best.



➤ Purpose:Stops training automatically when the model stops improving.

✅ Why use it?
1. To prevent overfitting (where your model learns the training data too well and does poorly on new data).
2. To save time — no need to train extra epochs when the model isn’t getting better!

____________________________________________________________________________________________________________________________

What is TensorBoard?
TensorBoard is a visual tool that helps you understand and debug your machine learning models.TensorBoard shows graphs, charts, and metrics (like accuracy, loss, etc.) while your model is training — in your web browser.It’s like a live report card for your model. TensorBoard is a browser-based tool to visualize model training in TensorFlow/Keras.It helps with	Debugging, monitoring, and improving model performance.It is common with callbacks=[TensorBoard(...)] in model.fit()



What Can TensorBoard Show You?

Here’s what you can visualize with TensorBoard:
Feature: 

Scalars---->Loss, accuracy, learning rate over time

Graphs---->Your model’s architecture (layers, connections)

Histograms---->Distributions of weights and biases

Images & Text---->If you’re working with images or NLP, you can see them too

Projector---->View word embeddings in 3D



In [116]:
(X_train.shape[1],)     ## This , (blank) means it is a single dimension and it has 12 inputs.

(12,)

In [117]:
## Build Our ANN Model
model= Sequential([
    Dense(64,activation='relu',input_shape= (X_train.shape[1],)), ## HL 1 connected with input layer. ##Here first I need to give my input and my hidden layer 1 
    Dense(32,activation='relu'),    ## HL 2
    Dense(1,activation='sigmoid')     ## Output layer
]
)

c:\Users\Vishal Vishwakarma\Downloads\ANN CLASSIFICATION\venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


* What is Dense(...)?
A Dense layer is a layer where every neuron is connected to every input or neuron from the previous layer. It's called fully connected.

* Dense(64, activation='relu', input_shape=(X_train.shape[1],))   ----->means
What Each Part Means:

1. Dense(64, ...) ----> This creates a Dense layer with 64 neurons.

2. activation='relu' ----> Each of those 64 neurons will use the ReLU activation function.

3. input_shape=(X_train.shape[1],) ----> This tells Keras the shape of the input data.

_____________________________________________________________________________________________
* 64 — Number of Neurons: This means: The first hidden layer will have 64 neurons. Each of them will receive input from every input feature.

* 2. activation='relu' — Activation Function
* ReLU stands for Rectified Linear Unit

* It changes each number like this:
​✅ ReLU makes the model better at learning complex patterns, and helps prevent some training problems.

_______________________________________________________________________________________________

✅ 3. input_shape=(X_train.shape[1],)
* X_train.shape[1] means: the number of input features (columns) in your training data

* Example: if your data has 10 input features, this becomes: input_shape=(10,)
⚠️ input_shape is only needed in the first layer to tell the model the shape of the input data.

Summary Table:
Code Piece:
1. Dense(64) -----> 64 neurons (nodes) in this layer.

2. activation='relu' ----> Use ReLU function to activate the neurons.

3. input_shape=(X_train.shape[1],) ----> Input has the same number of features as your training data.

________________________________________________________________________________________________

👉 What does the 1 mean in X_train.shape[1]?
      In Python and NumPy, shape tells you the dimensions of an array or dataset.
 Example: Suppose your X_train looks like this:
 It might have:
* 8000 rows (samples)
* 3 columns (features),   Then X_train.shape → (8000, 3)
_______________________________________________________________________________________________
✅ Now:
X_train.shape[0] → 8000 → number of rows (samples)

X_train.shape[1] → 3 → number of features (columns)

📌 So in input_shape=(X_train.shape[1],):
* You're telling Keras: “Each input has X_train.shape[1] features.”

* The comma in (X_train.shape[1],) makes it a tuple — required by Keras.
_____________________________________________________________________________________________________
Summary:
Code:
X_train.shape	----> Tuple with (number of samples, number of features)
X_train.shape[1]	----> Number of features (columns)
input_shape=(X_train.shape[1],)	----> Tell Keras the shape of each input row




In [118]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

Now in order to do the forward and the backward propogation, we need to compile this model.

In [119]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.1)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x0000023E82675440>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [120]:
## Compile the model
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])

Steps Explaination:
* import tensorflow ---->👉 This loads the TensorFlow library so you can use its functions for deep   learning.

* (Define the Optimizer) 

opt = tensorflow.keras.optimizers.Adam(learning_rate=0.1) ----> 👉 You're using the Adam optimizer, which helps the model learn by updating weights during training.
📌 learning_rate=0.1: Controls how big the steps are when updating weights. (0.1 is quite high—usually 0.001 is safer.)

* (Define the Loss Function)

loss = tensorflow.keras.losses.BinaryCrossentropy() ---->
👉 This defines how we measure the model's error for binary classification (like spam/not spam, yes/no).
📌 The model tries to minimize this loss during training.

*  (Compile the Model)

model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy']) ---->
👉 This gets your model ready to train by setting:
1. optimizer=opt: Uses the Adam optimizer.

2. loss="binary_crossentropy": Tells the model how to measure error.

3. metrics=['accuracy']: Tracks accuracy while training (how many predictions are correct).




In [ ]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

✅ Step-by-Step Explanation
🔸 Step 1: Import Callbacks:

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard  --->👉 These are tools that help control or monitor training:
* EarlyStopping stops training if the model stops improving.

* TensorBoard is for visualizing training progress (like loss, accuracy graphs).


🔸 Step 2: Create a Log Directory:

log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") ----> 👉 Creates a folder name (like logs/fit20250702-1945) where logs will be saved.
📌 datetime.now() adds the current date and time to avoid overwriting old logs.


🔸 What is log in log_dir?
In this line:

log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") ----> log is short for log files. These are saved records of what happens during model training—like: Loss, Accuracy, Learning rate, Histograms of weights, etc.

🔸 What is "logs/fit"?

* "logs" is the main folder. 

* "fit" is a subfolder (you can name it anything).

* (+) datetime... adds the current date & time to keep each training log unique.


📂 So the final folder path looks like:

logs/fit20250702-1950


🔸 Why do we need it? --> TensorBoard reads these logs to show nice visual graphs of how your model is learning.



Step 3: Set Up TensorBoard Callback:

tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1) ----> 👉 This tells TensorBoard to:

* Save training logs to log_dir.

* Track histograms of weights every epoch (with histogram_freq=1).





In [122]:
## Set up Early Stopping
early_stopping_callback= EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)


Step-by-Step Explanation:
1. EarlyStopping
👉 A callback that automatically stops training when the model stops improving.

2. monitor='val_loss'
👉 It watches the validation loss after each epoch.

3. patience=5
👉 If validation loss doesn’t improve for 5 epochs, it stops training.

4. restore_best_weights=True
👉 After stopping, it restores the best model weights (from the epoch with the lowest val_loss).

🧠 Summary:
Stops training early if your model stops getting better, and keeps the best version of the model.

________________________________________________________________________________________________________
✅ What is Early Stopping (in terms of loss and epochs)?
EarlyStopping is a technique to automatically stop training your model when it stops improving—so you don’t overfit and waste time. EarlyStopping stops training when the validation loss doesn't get better for a while. This prevents overfitting and saves time.


🔍 How does it work (step-by-step)?
* After each epoch, it checks the validation loss (not training loss).

* If the validation loss decreases → it saves the model and continues training.

* If the validation loss increases or stays the same for a few epochs, it waits (this is called patience).

* If no improvement happens for those many epochs → it stops training early.
________________________________________________________________________________________________________

What is Validation Loss?
Validation loss is the error your model makes on data it has never seen before — called the validation set.

🔍 In simple terms:
During training, you split your dataset like this:

* Training data → used to train the model (update weights).

* Validation data → used to check how well the model is generalizing.


📊 Loss types:
Loss Type:
Training Loss	---> Error on the training data.
Validation Loss	---> Error on the validation data.


🧠 Why is Validation Loss important?
If training loss ↓ but validation loss ↑, your model is overfitting — it’s memorizing instead of learning. We want both training loss and validation loss to go down.

📉 Example:
Epoch	Training Loss	Validation Loss
1	        0.60	          0.58
2	        0.50	          0.48
3	        0.40	          0.55 ❌

At Epoch 3, validation loss increases → model is overfitting → early stopping can help!
______________________________________________________________________________________________________

✅ What is Training Loss?
Training loss is the error (or difference) between the model's predictions and the actual labels on the training data.


🔍 In simple words:
It tells you how well your model is learning from the data you gave it during training.


📊 Example:
If you're training a model to detect cats vs. dogs:

* Your model predicts dog for an image of a cat → ❌ wrong → some loss is added.

* The model gets better over time → makes fewer mistakes → training loss decreases.


📉 Goal: We want the training loss to go down over epochs. That means the model is learning and improving on the training data.


🤔 But be careful: A very low training loss does not always mean a good model. It might be memorizing the training data → leading to overfitting. That’s why we also check validation loss.


🧠 Summary:

Term:	                What it Measures:	                                        Data Used:
Training Loss	        Model error on data it trained on	                     Training data
Validation Loss	        Model error on unseen data (checks generalization)	     Validation data





In [123]:
## Train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


c:\Users\Vishal Vishwakarma\Downloads\ANN CLASSIFICATION\venv\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 45ms/step - accuracy: 0.7869 - loss: 0.5378 - val_accuracy: 0.8035 - val_loss: 0.4848
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - accuracy: 0.7893 - loss: 0.4621 - val_accuracy: 0.8035 - val_loss: 0.3979
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.7876 - loss: 0.4698 - val_accuracy: 0.8035 - val_loss: 0.4296
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - accuracy: 0.7905 - loss: 0.4360 - val_accuracy: 0.8035 - val_loss: 0.4182
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - accuracy: 0.7964 - loss: 0.4207 - val_accuracy: 0.8035 - val_loss: 0.4086
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - accuracy: 0.7925 - loss: 0.4272 - val_accuracy: 0.8035 - val_loss: 0.4061
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.8001 - loss: 0.4008 - val_accuracy: 0.8035 - val_loss: 0.3937
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.8049 - loss: 0.4106 - val_accura

In [124]:
model.save('model.h5')

In [ ]:
## Load Tensorboard Extension
%load_ext tensorboard 

In [134]:
%tensorboard --logdir logs/fit20250703-003054

Reusing TensorBoard on port 6009 (pid 4772), started 0:07:08 ago. (Use '!kill 4772' to kill it.)

In [ ]:
## Load the pickle file
